In [ ]:
!pip install --upgrade networkx

In [ ]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import networkx as nx
from networkx.algorithms import bipartite

In [ ]:
def css_styling():
    styles = open('../../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

In [ ]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
from client.api.notebook import Notebook 
hwk_sw = Notebook('hw04.ok')
_ = hwk_sw.auth(inline=True)


# Homework 04

## Small worlds

In this homework assignment, we're going to explore the concept of *small worlds*, which we discussed in class earlier this semester.  Small worlds have long been studied by social networks researchers, and they have also been discussed in popular culture. As a reminder, the rough idea is that social networks can typically be expected to have two characteristics:

* a high level of clustering
* a short average path length

A high level of clustering is consistent with the idea of triadic closure. And a short average path length is supposed to capture situations we often seem to encounter in our day to day lives: e.g., two strangers find that they have an unexpected acquaintance in common and exclaim "it's a small world!" (see the Milgram article below).

We're going to try to assess how well these two small world predictions hold up empirically. We're going to focus on the Add Health networks. We should bear in mind that the small world theory is really about very large networks, so we will be evaluating it in an unusual situation: networks of moderate size taken from children who all live in the same community.

In case you want to read some of the original small world research papers, you can check out some of the papers we talked about in lecture. Here is an article describing an early empirical study by Milgram:

* [Milgram 1967](http://measure.igpp.ucla.edu/GK12-SEE-LA/Lesson_Files_09/Tina_Wey/TW_social_networks_Milgram_1967_small_world_problem.pdf)

And here are a couple of more recent studies in which researchers analyzed mathematical models that can produce networks with small-world properties:

* [Watts & Strogatz 1998](http://www.nature.com/nature/journal/v393/n6684/abs/393440a0.html)
* [Watts 1999](http://www.jstor.org/stable/10.1086/210318?seq=1#page_scan_tab_contents)

First, we'll start by looking at this sample network: 

In [ ]:
test_net = nx.Graph([(1,2), (1,3), (2,3), (4,5), (4,6), (4,3), (5,6), (3,5), (2,6), (7,8), (8,9)])
nx.draw_circular(test_net, with_labels=True)


**Question 1** Fill in the table below with the shortest distance between each pair of vertices. Enter INF if the nodes are disconnected

|             |  node 1 | node 2 |  node 3 | node 4 | node 5 | node 6 | node 7 | node 8 | node 9 |
|   :----:    |  :---:  |  :---: |  :---:  |  :---: |  :---: |  :---: |  :---: |  :---: |  :---: |
|   node 1    |    -    |  (?)   |    (?)  |    (?) |    (?) |    (?) |    (?) |    (?) |    (?) |
|   node 2    |    -    |    -   |    (?)  |    (?) |    (?) |    (?) |    (?) |    (?) |    (?) |
|   node 3    |    -    |    -   |    -    |    (?) |    (?) |    (?) |    (?) |    (?) |    (?) |
|   node 4    |    -    |    -   |    -    |   -    |    (?) |    (?) |    (?) |    (?) |    (?) |
|   node 5    |    -    |    -   |    -    |   -    |   -    |    (?) |    (?) |    (?) |    (?) |
|   node 6    |    -    |    -   |    -    |   -    |   -    |   -    |    (?) |    (?) |    (?) |
|   node 7    |    -    |    -   |    -    |   -    |   -    |        |   -    |    (?) |    (?) |
|   node 8    |    -    |    -   |    -    |   -    |   -    |        |        |   -    |    (?) |
|   node 9    |    -    |    -   |    -    |   -    |   -    |        |        |        |   -    |

<!-- <div class='solution'> -->
|             |  node 1 | node 2 |  node 3 | node 4 | node 5 | node 6 | node 7 | node 8 | node 9 |
|   :----:    |  :---:  |  :---: |  :---:  |  :---: |  :---: |  :---: |  :---: |  :---: |  :---: |
|   node 1    |    -    |   1    |    1    |    2   |    2   |    2   |    INF |    INF |    INF |
|   node 2    |    -    |    -   |    1    |    2   |    2   |    1   |    INF |    INF |    INF |
|   node 3    |    -    |    -   |    -    |    1   |    1   |    2   |    INF |    INF |    INF |
|   node 4    |    -    |    -   |    -    |   -    |    1   |    1   |    INF |    INF |    INF |
|   node 5    |    -    |    -   |    -    |   -    |   -    |    1   |    INF |    INF |    INF |
|   node 6    |    -    |    -   |    -    |   -    |   -    |   -    |    INF |    INF |    INF |
|   node 7    |    -    |    -   |    -    |   -    |   -    |        |   -    |    1   |     2  |
|   node 8    |    -    |    -   |    -    |   -    |   -    |        |        |   -    |    1   |
|   node 9    |    -    |    -   |    -    |   -    |   -    |        |        |        |   -    |


** Question 2** Now manually calculate the average shortest path length **for each** of the connected components in the network.


<div class='response'>
[Answer here]
</div>

<div class='solution'>
There are two connected components (CC) in the network
1) CC with nodes 1,2,3,4,5,6
Average shortest path length = (1+1+2+2+2+1+2+2+1+1+1+2+1+1+1)/15=1.4
2) CC with nodes 7,8,9
Average shortest path length = (1+2+1)/3 = 1.33
   </div>

** Question 3** Now verify the average shortest path length for each of the components in the graph, using the average_shortest_path_length method of the networkx library. For this purpose, one has to iterate over the connected components of the test_net graph.


In [ ]:
for ... in nx.connected_component_subgraphs(...):
    print (...(...)) # first ellipsis is for the function name, while the second one is for the arguments of the function

In [ ]:
#SOLUTION
for each in nx.connected_component_subgraphs(test_net):
    print (nx.average_shortest_path_length(each))

For rest of this homework, we'll use the code that we used in the labs to read the Add Health networks in.

In [ ]:
def read_add_health_network(network_id):
    """
    network_id : integer from 1 to 84
    
    read in the Add Health network corresponding to the given id number and
    return it as an undirected networkx object
    """

    # this file was downloaded from
    # http://moreno.ss.uci.edu/data.html#adhealth
    edge_file = os.path.join("../..", "data", "add-health", "comm" + str(network_id) + ".dat")
    with open(edge_file, 'r') as f:
        edge_lines = f.readlines()
        
    network = nx.parse_edgelist(edge_lines, nodetype=int, data=[('activity_level', float)])
    
    # note that we call the to_undirected method to ensure we get an undirected network
    return(network.to_undirected())

number_add_health_networks = 84
add_health_networks = [read_add_health_network(x) for x in range(1,number_add_health_networks+1)]

As a fist step in the analysis, we will create a helper function to calculate average degree of a network.

In [ ]:
def average_degree(net):
    return(... / ...)

In [ ]:
#SOLUTION
def average_degree(net):
    return(2 * net.number_of_edges() / net.number_of_nodes())

In [ ]:
avg_d=average_degree(test_net)
print (round(avg_d,2))

In [ ]:
_ = hwk_sw.grade('q3')

###  Empirical distribution in the Add Health networks

First, we'll look at the empirical distribution of clustering and average path length in the Add Health networks.

**Question 4** Write a loop that goes through each of the 84 Add Health networks and calculates the clustering coefficient and the number of nodes in the network. (Please use the average clustering coefficient, implemented by the `average_clustering` function from the networkx package.) Store the results in a Table called `add_health_clustering` using columns called `num_nodes` and `avg_clustering_coef`.

In [ ]:
clustering = make_array()
num_nodes = make_array()

for g in ...:
    clustering = np.append(clustering, ...)
    num_nodes = np.append(num_nodes, ...)
    
add_health_clustering = Table().with_columns(['num_nodes', num_nodes,
                                              'avg_clustering_coef', clustering])
add_health_clustering

In [ ]:
#SOLUTION
clustering = make_array()
num_nodes = make_array()

for g in add_health_networks:
    clustering = np.append(clustering, nx.average_clustering(g))
    num_nodes = np.append(num_nodes, g.number_of_nodes())
    
add_health_clustering = Table().with_columns(['num_nodes', num_nodes,
                                              'avg_clustering_coef', clustering])
add_health_clustering

In [ ]:
_ = hwk_sw.grade('q4')

**Question 5** Plot a histogram showing the distribution of clustering coefficients across the 84 Add Health networks.

In [ ]:
...

In [ ]:
#SOLUTION
add_health_clustering.hist('avg_clustering_coef')

**Question 6** Make a scatter plot that compares the number of nodes in each network (x axis) to the clustering coefficient (y axis). Does it look like the clustering coefficient changes as the number of nodes does?

In [ ]:
...

<div class='response'>
[Answer here]
</div>

In [ ]:
#SOLUTION
add_health_clustering.scatter('num_nodes', 'avg_clustering_coef')

<div class='solution'>
Yes: it looks like the average clustering coefficient decreases as the number of nodes increases. This decrease does not look linear: it is very sharp at first, and then it tapers off.
</div>

###  Average path length of biggest component

Remember that it really only makes sense to think about the average path length between two nodes that are in the same component. (Nodes in different components have no path between them.) Since some of the Add Health networks have more than one component, we'll start by picking out only the largest component in each network.

In [ ]:
def get_biggest_component(network):
    biggest = max(nx.connected_component_subgraphs(network), key=len)
    return(biggest)

add_health_biggest_components = [get_biggest_component(g) for g in add_health_networks]

**Question 7** Write a loop that goes through the largest component of each of the 84 Add Health networks and calculates the average shortest path length and the number of nodes in the network. Store the results in a Table called `add_health_sp` using columns called `num_nodes` and `avg_shortest_path`.

In [ ]:
## NOTE: your code might take a little while
##       (~3-5 minutes) to run

avg_shortest_path = make_array()
num_nodes = make_array()

avg_degree = make_array()

for c in ...:
    avg_shortest_path = np.append(avg_shortest_path, ...)
    num_nodes = np.append(num_nodes, ...)
    avg_degree = np.append(avg_degree, ...)

add_health_sp = Table().with_columns(['num_nodes', num_nodes,
                                      'avg_shortest_path', avg_shortest_path,
                                      'avg_degree', avg_degree])

add_health_sp

In [ ]:
#SOLUTION

## NOTE: your code might take a little while
##       (~3-5 minutes) to run

avg_shortest_path = make_array()
num_nodes = make_array()

avg_degree = make_array()

for c in add_health_biggest_components:
    avg_shortest_path = np.append(avg_shortest_path, nx.average_shortest_path_length(c))
    num_nodes = np.append(num_nodes, c.number_of_nodes())
    avg_degree = np.append(avg_degree, average_degree(c))

add_health_sp = Table().with_columns(['num_nodes', num_nodes,
                                      'avg_shortest_path', avg_shortest_path,
                                      'avg_degree', avg_degree])

add_health_sp

In [ ]:
_ = hwk_sw.grade('q7')

**Question 8** Plot a histogram showing the distribution of average shortest path lengths across the 84 Add Health networks' largest components.

In [ ]:
...

In [ ]:
#SOLUTION
add_health_sp.hist('avg_shortest_path')

**Question 9** Make a scatter plot that compares the number of nodes in each largest component (x axis) to the average shortest path (y axis). Does it look like the average shortest path changes as the number of nodes does?

In [ ]:
...

In [ ]:
#SOLUTION
add_health_sp.scatter('num_nodes', 'avg_shortest_path')

<div class='response'>
[Answer here]
</div>

<div class='solution'>
It looks like the average shortest path length increases as the number of nodes does. This increase does not look linear: it is very sharp when num_nodes is small. As num_nodes increases, there is more variation so it becomes harder to generalize; however, it looks like the increase in average shortest path length tends to get smaller.
</div>

### P-values

In the introduction to this section, you read that the small world theory suggests that a social network should have a large clustering coefficient and a small average path length. But what do large and small mean? In other words, what should we think about comparing these networks to?

We'll use Erdos-Renyi random networks as a null model. Specifically, we're going to

* pick one specific Add Health network to test
* generate ER networks that 'match' that specific Add Health network
* compare the clustering coefficient / average path lengths of the ER networks to the ones we observe in the Add Health network


Let's pick out one particular Add Health network to focus on for this part.

In [ ]:
# the specific Add Health network we'll look at
ahn = add_health_networks[17]

... and let's also use a couple of functions that we created in Lab 3.

In [ ]:
def er_by_degree(n, avg_degree):
    return(nx.erdos_renyi_graph(n=n, p=avg_degree / (n-1)))

def rand_er_network(network):
    """
    Return a random network generated from the configuration model using
    the degree sequence of the network passed in
    """
    network_n = network.number_of_nodes()
    network_dbar = average_degree(network)
    return(er_by_degree(network_n, network_dbar))

### Developing a simulation from a null model

**Question 10** Describe what the function `er_by_degree` does.  
*[HINT: You may find Lab 3 helpful to review.]*

<div class='response'>
[Answer here]
</div>

<div class='solution'>
`er_by_degree` generates an Erdos-Renyi random network with `n` nodes and expected degree of `avg_degree.`
</div>

**Question 11** Write a function which, given a network, returns its average shortest path length. If the network has more than one component, your function should return the average path length in the biggest component.

In [ ]:
def avg_path_length(net):
    if nx.number_connected_components(net) > 1:
        net = ...
    return(...)

In [ ]:
#SOLUTION
def avg_path_length(net):
    if nx.number_connected_components(net) > 1:
        net = get_biggest_component(net)
    return(nx.average_shortest_path_length(net))

In [ ]:
_ = hwk_sw.grade('q11')

**Question 12** Write a simulation that generates 100 Erdos Renyi random networks that match the Add Health network `ahn`. (By 'match', we mean that the ER network should have the same average degree and number of nodes as the Add Health network `ahn`.). For each generated ER network, calculate the average clustering and use the function you wrote above to calculate the average path length. Store the results in a table called `er_res`.

In [ ]:
observed_apl = avg_path_length(ahn)
observed_cc = nx.average_clustering(ahn)

er_cc = make_array()
er_apl = make_array()

# the underscore (_) means that we don't
# care which iteration is which -- we just want
# to repeat this 100 times
for _ in range(100):
    
    er_net = ...
    er_cc = np.append(er_cc, ...)
    er_apl = np.append(er_apl, ...)
    
er_res = Table().with_columns('cc', er_cc,
                              'apl', er_apl)

er_res

In [ ]:
#SOLUTION
observed_apl = avg_path_length(ahn)
observed_cc = nx.average_clustering(ahn)

er_cc = make_array()
er_apl = make_array()

# the underscore (_) means that we don't
# care which iteration is which -- we just want
# to repeat this 100 times
for _ in range(100):
    
    er_net = rand_er_network(ahn)
    er_cc = np.append(er_cc, nx.average_clustering(er_net))
    er_apl = np.append(er_apl, avg_path_length(er_net))
    
er_res = Table().with_columns('cc', er_cc,
                              'apl', er_apl)

er_res

In [ ]:
round(np.corrcoef(er_res['cc'], er_res['apl'])[0,1],1)

In [ ]:
np.mean(er_res.column(0))

In [ ]:
_ = hwk_sw.grade('q12')

**Question 13** Now print out the observed average path length in the Add Health network `ahn` and plot a histogram of the average path lengths in the ER networks you just simulated.

In [ ]:
...
...

In [ ]:
#SOLUTION
print("The observed average path length is:", observed_apl)
er_res.hist('apl')

**Question 14** Where would the observed Add Health network's statistic fall in the Erdos Renyi networks' distribution?

<div class='response'>
[Answer here]
</div>

###  Calculating P values

**Note**: Questions 15 and 17 below ask you to calculate a *p* value. The calculation is not especially tricky, but if you want to review *p* values and hypothesis tests in general, you can check out these [slides from Data 8](https://docs.google.com/presentation/d/1SXmBC3B452sW1qerhQ-bjpoR58tmAz100yf6rj2D1iI/edit#slide=id.g210ec578e3_0_0).

**Question 15** Now use your results to calculate a $p$ value for the hypothesis that the Add Health network's average path length was generated by the ER model; the alternative hypothesis should be that the Add Health network's average path length is larger than it would be in the ER model.  

In [ ]:
emp_p_value_apl=np.mean(...) #fill in the condition corresponding to the alternative hypothesis
emp_p_value_apl

In [ ]:
#SOLUTION
emp_p_value_apl = np.mean(observed_apl >= er_res.column('apl'))
emp_p_value_apl

In [ ]:
_ = hwk_sw.grade('q15')

**Question 16** Now print out the observed average clustering in the Add Health network `ahn` and plot a histogram of the average clusterings in the ER networks you just simulated. Look at where the observed Add Health network's statistic would fall in the ER distribution.

In [ ]:
...
...

In [ ]:
#SOLUTION
print("Observed average clustering coefficient:", observed_cc)
er_res.hist('cc')

**Question 17** Now use your results to calculate a $p$ value for the hypothesis that the Add Health network's average clustering was generated by the ER model; the alternative hypothesis should be that the Add Health network's average clustering is less than it would be in the ER model.

In [ ]:
emp_p_value_cc = np.mean(...) #fill in the condition corresponding to the alternative hypothesis
emp_p_value_cc

In [ ]:
#SOLUTION
emp_p_value_cc = np.mean(observed_cc <= er_res.column('cc'))
emp_p_value_cc

In [ ]:
_ = hwk_sw.grade('q17')

**Question 18** What do these two $p$ values lead you to conclude about the agreement between the ER model and the small world hypothesis (at least, using information from the Add Health network)?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
The first $p$ value suggests that the average path length observed in the Add Health network is at least as big as the average path length implied by the ER model.  
The second $p$ value suggests that the observed clustering in the Add Health network is higher than the average clustering implied by the ER model.
</div>

# SUBMIT YOUR ASSIGNMENT

In order to submit your assignment, run the next cell.

You can submit as many times as you want (up to the deadline).

In [ ]:
_ = hwk_sw.submit()